# Integrate Agenta with LangGraph (Openinference)

This notebook demonstrates how to connect **Agenta** with **LangGraph** for comprehensive observability and debugging of your graph-based LLM applications.

> **What is Agenta?** [Agenta](https://agenta.ai) is an open-source LLMOps platform designed to streamline the deployment, management, and scaling of large language models. It offers comprehensive observability, testing, and deployment capabilities for AI applications.

> **What is LangGraph?** [LangGraph](https://langchain-ai.github.io/langgraph/) is a library for building stateful, multi-actor applications with LLMs. It extends LangChain's capabilities by enabling the creation of complex workflows as directed graphs where nodes represent different processing steps and edges define the flow between them.

## Implementation Guide

Follow this tutorial to set up LangGraph with Agenta's observability platform for real-time application insights.

### Step 1: Install Required Dependencies

Install the necessary Python packages for this integration:

In [ ]:
!pip install agenta langchain langgraph langchain-openai langchain-community llama-index openinference-instrumentation-langchain

**Package Descriptions:**
- `agenta`: Core SDK for Agenta's prompt engineering and observability platform
- `langchain`: Framework for building LLM applications with chains and agents
- `langgraph`: Extension for creating graph-based LLM workflows
- `langchain-openai`: OpenAI integrations for LangChain
- `langchain-community`: Community extensions for LangChain
- `llama-index`: Document loading and processing utilities
- `openinference-instrumentation-langchain`: Automatic instrumentation library for LangChain operations

### Step 2: Setup and Configuration

Configure your environment and initialize the Agenta SDK:

In [ ]:
import os
import agenta as ag
from typing import TypedDict, Dict
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from llama_index.core import SimpleDirectoryReader
from langchain_core.runnables import RunnableLambda
from openinference.instrumentation.langchain import LangChainInstrumentor


# Load configuration from environment
os.environ["AGENTA_API_KEY"] = "your_agenta_api_key"
os.environ["AGENTA_HOST"] = (
    "https://cloud.agenta.ai"  # Optional, defaults to the Agenta cloud API
)
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"  # Required for OpenAI Agents SDK


# Start Agenta SDK
ag.init()

**What does `ag.init()` do?**
This function initializes the Agenta SDK and sets up the necessary configuration for observability. It establishes connection to the Agenta platform, configures tracing and logging settings, and prepares the instrumentation context for your application.

### Step 3: Enable LangChain Monitoring

Initialize the OpenInference LangChain instrumentation to automatically capture LangGraph operations:

In [ ]:
# Enable LangChain instrumentation (includes LangGraph)
LangChainInstrumentor().instrument()

### Step 4: Configure Language Model

Set up your language model for the LangGraph workflow:

In [ ]:
# Configure ChatOpenAI model
llm = ChatOpenAI(model="gpt-4", temperature=0)

### Step 5: Build Your Instrumented LangGraph Application

Here's a complete example showcasing a meeting transcript analysis workflow with Agenta instrumentation:

#### Define State Structure

In [ ]:
# Define state structure for the graph
class SummarizerState(TypedDict):
    input: str
    segments: Dict[str, list[str]]
    speaker_summaries: Dict[str, str]
    actions: str

#### Load Meeting Transcripts

In [ ]:
# Load meeting transcripts from documents
# Note: Create a 'meetings' directory with .txt or .md files for this to work
try:
    documents = SimpleDirectoryReader("meetings").load_data()
    full_transcript = "\n".join(doc.text for doc in documents)
except Exception:
    # Fallback sample transcript for demonstration
    full_transcript = """
John: Good morning everyone, let's start our weekly standup.
Sarah: I finished the user authentication feature yesterday.
Mike: I'm working on the database optimization, should be done by Friday.
John: Great! Sarah, can you help Mike with testing once he's done?
Sarah: Absolutely, I'll be available.
John: Perfect. Our action items are: Sarah to help Mike with testing, and Mike to finish database optimization by Friday.
""".strip()

print("Sample transcript loaded:", full_transcript[:100] + "...")

#### Define Graph Nodes

In [ ]:
# Node 1: Segment speaker contributions
def segment_by_speaker(state):
    transcript = state["input"]
    speakers = {}
    for line in transcript.split("\n"):
        if ":" in line:
            name, text = line.split(":", 1)
            speakers.setdefault(name.strip(), []).append(text.strip())
    return {**state, "segments": speakers}


# Node 2: Summarize each speaker's contributions
def summarize_per_speaker(state):
    segments = state["segments"]
    summaries = {}
    for speaker, texts in segments.items():
        joined_text = " ".join(texts)
        summary = llm.invoke(f"Summarize what {speaker} said: {joined_text}")
        summaries[speaker] = summary.content
    return {**state, "speaker_summaries": summaries}


# Node 3: Extract action items
def extract_actions(state):
    transcript = state["input"]
    result = llm.invoke(f"List all action items from this transcript:\n{transcript}")
    return {**state, "actions": result.content}

#### Create Instrumented Analysis Function

In [ ]:
@ag.instrument()
def meeting_analyzer(transcript: str):
    # Build LangGraph workflow
    builder = StateGraph(SummarizerState)
    builder.add_node("segment", RunnableLambda(segment_by_speaker))
    builder.add_node("summarize", RunnableLambda(summarize_per_speaker))
    builder.add_node("extract_actions", RunnableLambda(extract_actions))

    builder.set_entry_point("segment")
    builder.add_edge("segment", "summarize")
    builder.add_edge("summarize", "extract_actions")
    builder.add_edge("extract_actions", END)

    graph = builder.compile()
    result = graph.invoke({"input": transcript})
    return result

#### Run the Analysis

In [ ]:
# Example usage
result = meeting_analyzer(full_transcript)
print("Analysis Result:")
print("Segments:", result["segments"])
print("\nSpeaker Summaries:", result["speaker_summaries"])
print("\nAction Items:", result["actions"])

### Step 6: Understanding the @ag.instrument() Decorator

The `@ag.instrument()` decorator automatically captures all input and output data from your function, enabling comprehensive observability without manual instrumentation.

**Span Type Configuration:**
Use the `spankind` parameter to categorize operations in Agenta WebUI. Available options:

- `agent` - Autonomous agent behaviors
- `chain` - Sequential processing workflows
- `workflow` - Complete application processes (default)
- `tool` - Utility and helper functions
- `embedding` - Vector embedding operations
- `query` - Search and retrieval tasks
- `completion` - Text generation operations
- `chat` - Conversational interfaces
- `rerank` - Result ordering operations

**Standard Behavior:**
By default, when `spankind` is not specified, the operation becomes a root-level span, categorized as a `workflow` in Agenta.

In [ ]:
# Example with custom span classification:
@ag.instrument(spankind="chain")
def document_processing_chain(documents: list):
    # Document processing workflow
    pass

### Step 7: View Traces in Agenta

After running your application, access detailed execution traces through Agenta's dashboard. The observability data includes:

- Complete graph workflow execution timeline
- Individual node processing steps and state transitions
- LLM invocations and response generation
- State updates and data flow between nodes
- Document loading and preprocessing operations
- Performance metrics and timing analysis


<img 
    style="display: block; margin: 20px; text-align: center"
    src="./images/agenta-openinference-langgraph-trace.png"
    width="90%"
    alt="Agenta dashboard showing LangGraph application trace with detailed execution steps">


The observability interface provides insights for:
- Debug complex graph workflows and state management
- Monitor node execution performance and bottlenecks
- Analyze LLM usage patterns and token consumption
- Track data flow and state transitions between nodes

## Advanced Usage

### Custom Span Configuration

Customize instrumentation for different application components:

In [ ]:
@ag.instrument(spankind="workflow")
def document_analysis_pipeline(file_path: str):
    return meeting_analyzer(file_path)


@ag.instrument(spankind="tool")
def custom_document_loader(directory: str):
    # Custom document loading logic
    pass


@ag.instrument(spankind="chain")
def multi_step_analysis(transcript: str):
    # Multi-step analysis workflow
    return transcript

### Real-world Examples

#### Customer Feedback Analysis System

In [ ]:
class FeedbackState(TypedDict):
    input: str
    sentiment: str
    categories: list[str]
    priority: str
    response_draft: str


def analyze_sentiment(state):
    feedback = state["input"]
    result = llm.invoke(f"Analyze sentiment of this feedback: {feedback}")
    return {**state, "sentiment": result.content}


def categorize_feedback(state):
    feedback = state["input"]
    result = llm.invoke(f"Categorize this feedback into relevant topics: {feedback}")
    return {**state, "categories": result.content.split(", ")}


def determine_priority(state):
    sentiment = state["sentiment"]
    categories = state["categories"]
    result = llm.invoke(
        f"Determine priority (high/medium/low) based on sentiment: {sentiment} and categories: {categories}"
    )
    return {**state, "priority": result.content}


def draft_response(state):
    feedback = state["input"]
    sentiment = state["sentiment"]
    result = llm.invoke(
        f"Draft a professional response to this {sentiment} feedback: {feedback}"
    )
    return {**state, "response_draft": result.content}


@ag.instrument(spankind="workflow")
def feedback_processor(feedback_text: str):
    builder = StateGraph(FeedbackState)
    builder.add_node("sentiment", RunnableLambda(analyze_sentiment))
    builder.add_node("categorize", RunnableLambda(categorize_feedback))
    builder.add_node("priority", RunnableLambda(determine_priority))
    builder.add_node("response", RunnableLambda(draft_response))

    builder.set_entry_point("sentiment")
    builder.add_edge("sentiment", "categorize")
    builder.add_edge("categorize", "priority")
    builder.add_edge("priority", "response")
    builder.add_edge("response", END)

    graph = builder.compile()
    return graph.invoke({"input": feedback_text})

#### Test Customer Feedback Analysis

In [ ]:
# Test feedback analysis
sample_feedback = "Your product is amazing but the customer service was terrible. I waited 2 hours on hold!"
feedback_result = feedback_processor(sample_feedback)
print("Feedback Analysis Result:")
print("Sentiment:", feedback_result["sentiment"])
print("Categories:", feedback_result["categories"])
print("Priority:", feedback_result["priority"])
print("Response Draft:", feedback_result["response_draft"])

#### Research Paper Analysis Pipeline

In [ ]:
class ResearchState(TypedDict):
    input: str
    abstract_summary: str
    key_findings: list[str]
    methodology: str
    limitations: str
    relevance_score: float


@ag.instrument(spankind="chain")
def research_analyzer(paper_text: str):
    def extract_abstract(state):
        paper = state["input"]
        result = llm.invoke(
            f"Extract and summarize the abstract from this research paper: {paper}"
        )
        return {**state, "abstract_summary": result.content}

    def identify_findings(state):
        paper = state["input"]
        result = llm.invoke(f"List the key findings from this research paper: {paper}")
        return {**state, "key_findings": result.content.split("\n")}

    def analyze_methodology(state):
        paper = state["input"]
        result = llm.invoke(f"Describe the methodology used in this research: {paper}")
        return {**state, "methodology": result.content}

    def assess_limitations(state):
        paper = state["input"]
        result = llm.invoke(f"Identify limitations mentioned in this research: {paper}")
        return {**state, "limitations": result.content}

    def score_relevance(state):
        abstract = state["abstract_summary"]
        result = llm.invoke(
            f"Rate the relevance of this research on a scale of 0-10: {abstract}"
        )
        try:
            score = float(result.content.strip())
        except Exception:
            score = 5.0
        return {**state, "relevance_score": score}

    builder = StateGraph(ResearchState)
    builder.add_node("abstract", RunnableLambda(extract_abstract))
    builder.add_node("findings", RunnableLambda(identify_findings))
    builder.add_node("methodology", RunnableLambda(analyze_methodology))
    builder.add_node("limitations", RunnableLambda(assess_limitations))
    builder.add_node("relevance", RunnableLambda(score_relevance))

    builder.set_entry_point("abstract")
    builder.add_edge("abstract", "findings")
    builder.add_edge("findings", "methodology")
    builder.add_edge("methodology", "limitations")
    builder.add_edge("limitations", "relevance")
    builder.add_edge("relevance", END)

    graph = builder.compile()
    return graph.invoke({"input": paper_text})

#### Content Moderation Workflow

In [ ]:
class ModerationState(TypedDict):
    input: str
    toxicity_score: float
    content_categories: list[str]
    action_required: str
    explanation: str


@ag.instrument(spankind="workflow")
def content_moderator(user_content: str):
    def assess_toxicity(state):
        content = state["input"]
        result = llm.invoke(f"Rate toxicity of this content from 0-10: {content}")
        try:
            score = float(result.content.strip())
        except Exception:
            score = 0.0
        return {**state, "toxicity_score": score}

    def categorize_content(state):
        content = state["input"]
        result = llm.invoke(
            f"Categorize this content (spam, harassment, hate speech, etc.): {content}"
        )
        return {**state, "content_categories": result.content.split(", ")}

    def determine_action(state):
        toxicity = state["toxicity_score"]
        categories = state["content_categories"]
        if toxicity > 7:
            action = "remove"
        elif toxicity > 4:
            action = "flag_for_review"
        else:
            action = "approve"
        explanation = (
            f"Decision based on toxicity score: {toxicity} and categories: {categories}"
        )
        return {**state, "action_required": action, "explanation": explanation}

    builder = StateGraph(ModerationState)
    builder.add_node("toxicity", RunnableLambda(assess_toxicity))
    builder.add_node("categorize", RunnableLambda(categorize_content))
    builder.add_node("action", RunnableLambda(determine_action))

    builder.set_entry_point("toxicity")
    builder.add_edge("toxicity", "categorize")
    builder.add_edge("categorize", "action")
    builder.add_edge("action", END)

    graph = builder.compile()
    return graph.invoke({"input": user_content})

#### Test Content Moderation

In [ ]:
# Test content moderation
sample_content = "This is a great product, I highly recommend it to everyone!"
moderation_result = content_moderator(sample_content)
print("Content Moderation Result:")
print("Toxicity Score:", moderation_result["toxicity_score"])
print("Categories:", moderation_result["content_categories"])
print("Action Required:", moderation_result["action_required"])
print("Explanation:", moderation_result["explanation"])

## Next Steps

For more detailed information about Agenta's observability features and advanced configuration options, visit the [Agenta Observability SDK Documentation](/observability/observability-sdk).